# Сборный проект 2: Определение потребительской лояльности среди клиентов для телекоммуникационной компании.


<a id="num1"></a>
## Цели исследования

1. Определить текущий уровень потребительской лояльности, или NPS среди клиентов из России. 
2. Оформить результаты в визуальном представлении. 

<a id="num2"></a>
## Поставленные вопросы

1. Как распределены участники опроса: 
    - Каких пользователей больше по возрасту;
    - Каких пользователей больше по полу и возрасту; 
    - Каких пользователей больше: новых или старых? 
    - Пользователи из каких городов активнее участвовали в опросе?
2. Какие группы пользователей наиболее и наименее лояльны к сервису?
3. Какой общий NPS среди всех опрошенных?
4. Как можно описать клиентов, которые относятся к группе cторонников?

<a id="num3"></a>
## Оглавление 
* [1. Цели исследования](#num1)
* [2. Поставленные вопросы](#num2)
* [3. Оглавление](#num3)
* [4. Ключ к данным](#num4)
* [5. Подготовка данных](#num5) 
* [6. Исследование](#num6)

<a id="num4"></a>
## Ключ к данным 

<div style="background-color: #f2f2f2; color: #000; font-size: 14px; padding: 10px; margin: 10px; border: solid 1px #ccc; width: 98%; height: 98%">

  
| Название столбца | Описание |
|------------------|----------|
| user_id          | Идентификатор клиента |
| lt_day           | Количество дней «жизни» клиента |
| is_new	       | Поле хранит информацию о том, является ли клиент новым |
| age              | Возраст клиента в годах |
| gender_segment   | Пол клиента |
| os_name	       | Тип операционной системы |
| cpe_type_name	   | Тип устройства |
| country          | Страна проживания |
| city	           | Город проживания |
| age_segment	   | Возрастной сегмент |
| traffic_segment  | Сегмент по объёму потребляемого трафика |
| lifetime_segment | Сегмент по количеству дней «жизни» |
| nps_score        | Оценка клиента в NPS-опросе |
| nps_group        | Поле хранит информацию о том, к какой группе относится оценка клиента в опросе |

</div>


<a id="num5"></a>
## Подготовка данных

In [1]:
import os
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
    WITH data_cte AS (
    SELECT
        u.user_id,
        u.lt_day,
        u.age,
        u.gender_segment,
        u.os_name,
        u.cpe_type_name,
        l.country,
        l.city,
        SUBSTR(a.title, 3) AS age_segment,
        SUBSTR(t.title, 3) AS traffic_segment,
        SUBSTR(lt.title, 3) AS lifetime_segment,
        u.nps_score,
        CASE WHEN u.nps_score >= 9 THEN 'сторонники'
             WHEN u.nps_score >= 7 THEN 'нейтралы'
             ELSE 'критики'
        END AS nps_group,
        CASE WHEN u.lt_day <= 365 THEN 'True' ELSE 'False' END AS is_new
    FROM
        user AS u
        LEFT JOIN location AS l ON l.location_id = u.location_id
        LEFT JOIN age_segment AS a ON a.age_gr_id = u.age_gr_id
        LEFT JOIN traffic_segment AS t ON t.tr_gr_id = u.tr_gr_id
        LEFT JOIN lifetime_segment AS lt ON lt.lt_gr_id = u.lt_gr_id
)
SELECT *
FROM data_cte;
"""

In [4]:
df = pd.read_sql(query, engine)

In [5]:
df.head(10)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group,is_new
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники,False
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники,False
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники,False
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,сторонники,False
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,сторонники,False
5,A005O0,5501,42.0,1.0,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,критики,False
6,A0061R,1236,45.0,0.0,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,сторонники,False
7,A009KS,313,35.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,сторонники,True
8,A00AES,3238,36.0,1.0,ANDROID,SMARTPHONE,Россия,СанктПетербург,35-44,1-5,36+,10,сторонники,False
9,A00F70,4479,54.0,1.0,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,сторонники,False


In [6]:
df.to_csv('telecomm_csi_14_columns_v2.csv', index=False)

<a id="num6"></a>
## Исследование

В ходе исследования была оформлена презентация через Tableau. 

<a href="https://public.tableau.com/app/profile/a.ye5315/viz/2_16879751828050/sheet19?publish=yes">Сборный проект 2: Потребительская лояльность для телекоммуникационной компании.</a>